 ## 엑셀 데이터는 알집 파일에서 받을 수 있습니다.

In [1]:
cd C:\Users\USER\퀀트전략초보

C:\Users\USER\퀀트전략초보


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import dateutil.relativedelta as relativedelta 
warnings.filterwarnings(action = 'ignore')

 ### Multi Columns Concept

In [ ]:
Random01 = np.random.normal(0,1,size = (5,5))
Random48 = np.random.normal(4,8,size = (5,5))
big_col = ['Random01'] * 5 + ['Random48'] * 5
small_col = ['A','B','C','D','E']*2
df1 = pd.DataFrame(Random01)
df2 = pd.DataFrame(Random48)
df = pd.concat([df1,df2],axis = 1)
df.columns = [big_col , small_col]

In [ ]:
df

In [ ]:
df['Random01']

 ### Data Preprocessing

In [ ]:
path = '재무데이터.xlsx'
path_price = '수정주가.csv'
path_mkt = '시총데이터.csv'
path_endmonth = '결산월.csv'
path_kospiyn = '거래소데이터.csv'
path_delist_and_stop = '상장폐지데이터.csv'

 #### 재무 데이터 전처리 ####

In [ ]:
df = pd.read_excel(path, header = [0,1], index_col = 0)
cleaned_data = df.round(4)

In [ ]:
cleaned_data[['당기순이익','유동자산','총자본']].head()

 #### 가격 데이터 전처리 ####

In [ ]:
x = pd.DataFrame([])
for i in pd.read_csv(path_price, chunksize = 5000, parse_dates = ['Symbol'], index_col = 0) :
    x = pd.concat([x,i], axis = 0)
cleaned_price = x.astype(np.float64)

In [ ]:
cleaned_price.tail()

 #### 시가총액 데이터 전처리 ####

In [ ]:
mkt_data = pd.DataFrame([])
for i in pd.read_csv(path_mkt , index_col = 0, chunksize = 12000, engine = 'python') :
    mkt_data = pd.concat([mkt_data, i], axis = 0)

Common = mkt_data[mkt_data.columns[::2]].iloc[1:].applymap(lambda x : x.replace(',','') if type(x) == str else x).astype(np.float64)
Prefer = mkt_data[mkt_data.columns[1::2]].iloc[1:].applymap(lambda x : x.replace(',','') if type(x) == str else x).astype(np.float64)

mkt_data = pd.DataFrame(np.array(Common)/100 + np.array(Prefer.fillna(0))/100, columns = Common.columns, index = Common.index)
mkt_data.index = pd.to_datetime(mkt_data.index)
cleaned_mkt = mkt_data.resample('D').last().fillna(method = 'ffill', limit = 10) ### 중간에 공휴일도 가격을 보기 위해서 10개만 ffill

In [ ]:
cleaned_mkt.tail()

  #### 유가증권시장 or 코스닥 상장여부 전처리 ####

In [ ]:
mkt='유가증권시장'

In [ ]:
#########################################
data = pd.DataFrame([])
for i in pd.read_csv(path_kospiyn, index_col = 0 , chunksize = 6000 , engine = 'python') :
    data = pd.concat([data,i], axis = 0)
data.index = pd.to_datetime(data.index)
if mkt == '유가증권시장' or mkt == '코스닥':
    data = data.astype(str).applymap(lambda x : int(mkt in x)).resample('D').last()
elif mkt == 'both' :
    data = data.astype(str).applymap(lambda x : int('유가증권시장' in x or '코스닥' in x)).resample('D').last()
cleaned_kospiyn = data.fillna(method = 'ffill', limit = 30)

In [ ]:
cleaned_kospiyn.tail()

 #### 거래정지 및 상장폐지 데이터 전처리

In [ ]:
data = pd.DataFrame([])
for i in pd.read_csv(path_delist_and_stop, index_col = 0 , chunksize = 6000, engine = 'python') :
    data = pd.concat([data,i], axis = 0)
delist_data = data[[data.columns[1]]].iloc[1:]
delist_data = delist_data[delist_data['상장폐지일자'].isna() == False]
delist_data['상장폐지일자'] = delist_data['상장폐지일자'].apply(lambda x : pd.to_datetime(x))
small_col = list(data[data.columns[2:]].iloc[0])
small_col = pd.to_datetime(small_col)
big_col = pd.Series(data.columns[2:]).apply(lambda x : x.split('.')[0])    
stop_data = data[data.columns[2:]].iloc[1:]
stop_data.columns = [big_col,small_col]
stop_data = stop_data.applymap(lambda x : 1 if x in ['TRUE' , True] else 0)   

In [ ]:
stop_data.tail()

In [ ]:
delist_data.head()

 #### 결산월 전처리

In [ ]:
preprocessing_endmonth = lambda path_endmonth : pd.read_csv(path_endmonth,index_col = 0, parse_dates=['Symbol']).fillna(method = 'ffill').fillna(method = 'bfill').fillna(12)

In [ ]:
preprocessing_endmonth(path_endmonth).tail()

 #### 문제 : 지금까지 만든 전처리과정을 함수화하시오. (결산월 처리는 이미 위에 되어있으니 제외)

In [ ]:
def preprocessing_path(path) :
    ############################
    ## 재무 데이터 전처리     ##
    ############################

    return

def preprocessing_price(path_price) :
    ############################
    ## 가격 데이터 전처리     ##
    ############################

    return 

def preprocessing_mktdata(path_mkt_data) :    
    ############################
    ## 시가총액 데이터 전처리 ##
    ############################
    return
    
def preprocessing_kospiyn(path_kospiyn, mkt='유가증권시장') :
    ##########################################
    ### 거래소(코스피,코스닥)데이터 전처리  ##
    #########################################
    return 

def preprocessing_stop_and_delist(path_delist_and_stop) :
    #####################################################################################################
    ## 거래정지 및 상장폐지 데이터 전처리  결과값은 정지된 데이터와 상장폐지된 목록 두가지를 리턴하시오##
    #####################################################################################################    
    return  delist_data, stop_data

preprocessing_period = lambda path_endmonth : pd.read_csv(path_endmonth,index_col = 0, parse_dates=['Symbol']).fillna(method = 'ffill').fillna(method = 'bfill').fillna(12)